<a href="https://colab.research.google.com/github/derek-shing/ProofOfConcept/blob/master/Speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Testing the Speech to Text Cantonese API


1.   Preprocessing the audio file


*   How to record sound
*   turn to correct format
*   display codec spec.
*   Save to Storage




2.   Using AutoML API
*    Read from Storage
*    Call the API


https://cloud.google.com/speech-to-text/docs/quickstart  






In [0]:
project_id = 'my-project-1558599394309'

In [0]:
from google.colab import auth
auth.authenticate_user()

In [8]:
!gcloud config set project {project_id}

Updated property [core/project].


In [9]:
!gsutil cp gs://food1430/cantonese_flac.flac cantonese_flac.flac

Copying gs://food1430/cantonese_flac.flac...
/ [1 files][  3.3 MiB/  3.3 MiB]                                                
Operation completed over 1 objects/3.3 MiB.                                      


In [13]:
!ffmpeg -i cantonese_flac.flac -ac 1 mono.flac

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [1]:
!wget https://www.signalogic.com/melp/eng_noise/orig/car.wav

--2020-01-24 08:03:49--  https://www.signalogic.com/melp/eng_noise/orig/car.wav
Resolving www.signalogic.com (www.signalogic.com)... 209.150.126.178
Connecting to www.signalogic.com (www.signalogic.com)|209.150.126.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256044 (250K) [audio/x-wav]
Saving to: ‘car.wav’

car.wav             100%[===================>] 250.04K  --.-KB/s    in 0.06s   

2020-01-24 08:03:49 (4.41 MB/s) - ‘car.wav’ saved [256044/256044]



In [5]:
!gsutil cp /content/car.wav gs://food1430/car.wav

Copying file:///content/car.wav [Content-Type=audio/x-wav]...
-
Operation completed over 1 objects/250.0 KiB.                                    


In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/My Drive/GCP_account.json"

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
pip install google-cloud-speech

     |████████████████████████████████| 92kB 2.6MB/s 


In [14]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

# Instantiates a client
client = speech.SpeechClient()

# The name of the audio file to transcribe
#file_name = os.path.join(
#    os.path.dirname(__file__),
#    'resources',
#    'audio.raw')

file_name='/content/mono.flac'

# Loads the audio into memory
with io.open(file_name, 'rb') as audio_file:
    content = audio_file.read()
    audio = types.RecognitionAudio(content=content)

config = types.RecognitionConfig(
    encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
    sample_rate_hertz=44100,
    language_code='yue-Hant-HK')

# Detects speech in the audio file
response = client.recognize(config, audio)

for result in response.results:
    print('Transcript: {}'.format(result.alternatives[0].transcript))

Transcript: 工聯會區議員
Transcript: 荃灣一墜落下載
Transcript: 調動飛車問題
Transcript: 關全車嚟催淚煙清潔問題
Transcript: 他又提到有去年十二月有二十多人沖尤其辦事處破壞及偷竊
Transcript: 關注議員市民及前線警員安全問題
